In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from train_checkpoint import get_starter_dataset
from util.util import calculate_accuracy,compute_losses,simple_mia,standard_mia,plot_confusion_matrix
from torchvision.models import resnet18
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Checkpoints Info

In [3]:
retain_loader, forget_loader, val_loader,test_loader,train_loader = get_starter_dataset()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
n_class = 10
net = resnet18(weights=None, num_classes=n_class)
net.load_state_dict(torch.load('checkpoints/pre-train-model_epoch_40_lr_0.1_momentum_0.9_weightdecay_0.0005.pth'))
net.to(DEVICE);
print(f"Pre-trained performance, Retain_loader Accuracy: {calculate_accuracy(net,retain_loader):.2f}%, forget_loader Accuracy: {calculate_accuracy(net,forget_loader):.2f}%, Test_loader Accuracy: {calculate_accuracy(net,test_loader):.2f}%")
print("Pre-trained Standard MIA {:.2f}".format(standard_mia(net,forget_loader,test_loader)))

In [ ]:
n_class = 10
renet = resnet18(weights=None, num_classes=n_class)
renet.load_state_dict(torch.load('checkpoints/retrain-model_epoch_40_lr_0.1_momentum_0.9_weightdecay_0.0005.pth'))
renet.to(DEVICE);
print(f"Re-trained performance, Retain_loader Accuracy: {calculate_accuracy(renet,retain_loader):.2f}%, forget_loader Accuracy: {calculate_accuracy(renet,forget_loader):.2f}%, Test_loader Accuracy: {calculate_accuracy(renet,test_loader):.2f}%")
print("Re-trained Standard MIA {:.2f}".format(standard_mia(renet,forget_loader,test_loader)))

In [4]:
n_class = 10
renet = resnet18(weights=None, num_classes=n_class)
renet.load_state_dict(torch.load('checkpoints/scrub/scrub-model_epoch_10_lr_0.006_temp_2.pth'))
renet.to(DEVICE);
print(f"SCRUB performance, Retain_loader Accuracy: {calculate_accuracy(renet,retain_loader):.2f}%, forget_loader Accuracy: {calculate_accuracy(renet,forget_loader):.2f}%, Test_loader Accuracy: {calculate_accuracy(renet,test_loader):.2f}%")
print("SCRUB Standard MIA {:.2f}".format(standard_mia(renet,forget_loader,test_loader)))

SCRUB performance, Retain_loader Accuracy: 1.00%, forget_loader Accuracy: 0.70%, Test_loader Accuracy: 0.70%
SCRUB Standard MIA 0.51


In [4]:
n_class = 10
renet = resnet18(weights=None, num_classes=n_class)
renet.load_state_dict(torch.load('checkpoints/finetuning/finetuning-model_epoch_6_lr_0.1.pth'))
renet.to(DEVICE);
print(f"Finetune unlearning performance, Retain_loader Accuracy: {calculate_accuracy(renet,retain_loader):.2f}%, forget_loader Accuracy: {calculate_accuracy(renet,forget_loader):.2f}%, Test_loader Accuracy: {calculate_accuracy(renet,test_loader):.2f}%")
print("Finetune unlearning Standard MIA {:.2f}".format(standard_mia(renet,forget_loader,test_loader)))

Finetune unlearning performance, Retain_loader Accuracy: 1.00%, forget_loader Accuracy: 0.92%, Test_loader Accuracy: 0.77%
Finetune unlearning Standard MIA 0.58


# Data Info

In [ ]:
from util.util import label_distribution
print('data distribution: forget-retain-val-test')
label_distribution(forget_loader),label_distribution(retain_loader),label_distribution(val_loader),label_distribution(test_loader)

In [ ]:
plot_confusion_matrix(renet.to('cpu'), forget_loader)

## Embedding

In [ ]:
from util.emb import extract_data_by_target,extract_feature
import torch.nn as nn
import torch.nn.functional as F
renet.to(DEVICE)
pre_extractor = nn.Sequential(*list(net.children())[:-1])
re_extractor = nn.Sequential(*list(renet.children())[:-1])

forget9 = extract_data_by_target(forget_loader,3).to(DEVICE)
test9 = extract_data_by_target(test_loader,3).to(DEVICE)
emb_f9 = pre_extractor(forget9).squeeze()
emb_test9 = pre_extractor(test9).squeeze()
prob_f9 = F.softmax(net(forget9),dim=1)
prob_test9 = F.softmax(net(test9),dim=1)

reemb_f9 = re_extractor(forget9).squeeze()
reemb_test9 = re_extractor(test9).squeeze()
reprob_f9 = F.softmax(renet(forget9),dim=1)
reprob_test9 = F.softmax(renet(test9),dim=1)

In [ ]:
emb_f9 = pre_extractor(forget_loader).squeeze()
emb_test9 = pre_extractor(test_loader).squeeze()
prob_f9 = F.softmax(net(forget_loader),dim=1)
prob_test9 = F.softmax(net(test_loader),dim=1)
# pre-train forget vs retrain forget
cosine_similarities = F.cosine_similarity(emb_f9, reemb_f9, dim=1)
t_dist = F.pairwise_distance(emb_f9, reemb_f9)
print('embedding space cos smi and l2 distance between pre-train forget - retrain forget',cosine_similarities.mean(), t_dist.mean())

pcosine_similarities = F.cosine_similarity(prob_f9, reprob_f9, dim=1)
pt_dist = F.pairwise_distance(prob_f9, reprob_f9)
print('prob space cos smi and l2 distance between pre-train forget - retrain forget',pcosine_similarities.mean(), pt_dist.mean())

In [ ]:
# pre-train test vs retrain test
cosine_similarities = F.cosine_similarity(emb_test9, reemb_test9, dim=1)
t_dist = F.pairwise_distance(emb_test9, reemb_test9)
print('embedding space cos smi and l2 distance between pre-train test vs retrain test',cosine_similarities.mean(), t_dist.mean())

pcosine_similarities = F.cosine_similarity(prob_test9, reprob_test9, dim=1)
pt_dist = F.pairwise_distance(prob_test9, reprob_test9)
print('prob space cos smi and l2 distance between pre-train test vs retrain test',pcosine_similarities.mean(), pt_dist.mean())

In [ ]:
# pre-train forget vs pre-train test
lens = len(emb_f9) if len(emb_f9) < len(emb_test9) else len(emb_test9)
cosine_similarities = F.cosine_similarity(emb_f9[0:lens], emb_test9[0:lens], dim=1)
t_dist = F.pairwise_distance(emb_f9[0:lens], emb_test9[0:lens])
print('embedding space cos smi and l2 distance between pre-train forget vs pre-train test',cosine_similarities.mean(), t_dist.mean())

pcosine_similarities = F.cosine_similarity(prob_f9[0:lens], prob_test9[0:lens], dim=1)
pt_dist = F.pairwise_distance(prob_f9[0:lens], prob_test9[0:lens])
print('prob space cos smi and l2 distance between pre-train forget vs pre-train test',pcosine_similarities.mean(), pt_dist.mean())

In [ ]:
# retrain forget vs retrain test
lens = len(reemb_f9) if len(reemb_f9) < len(reemb_test9) else len(reemb_test9)
cosine_similarities = F.cosine_similarity(reemb_f9[0:lens], reemb_test9[0:lens], dim=1)
t_dist = F.pairwise_distance(reemb_f9[0:lens], reemb_test9[0:lens])
print('embedding space cos smi and l2 distance between retrain forget vs retrain test',cosine_similarities.mean(), t_dist.mean())


pcosine_similarities = F.cosine_similarity(reprob_f9[0:lens], reprob_test9[0:lens], dim=1)
pt_dist = F.pairwise_distance(reprob_f9[0:lens], reprob_test9[0:lens])
print('prob space cos smi and l2 distance between retrain forget vs retrain test',pcosine_similarities.mean(), pt_dist.mean())

In [ ]:
nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean()